# Exercise 2

bla bla

In [22]:
import bz2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [23]:
! if ! [[ -f data/amazon/train.ft.txt.bz2 ]]; then curl https://nilsreiter.de/assets/2020-08-31-deep-learning/amazon/train.ft.txt.bz2 > data/amazon/train.ft.txt.bz2; fi

In [24]:
! if ! [[ -f data/amazon/test.ft.txt.bz2 ]]; then curl https://nilsreiter.de/assets/2020-08-31-deep-learning/amazon/test.ft.txt.bz2 > data/amazon/test.ft.txt.bz2; fi

In [25]:
def get_labels_and_texts(file, limit=100000):
    labels = []
    texts = []
    lineNumber = 0
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
        lineNumber = lineNumber + 1
        if lineNumber >= limit and limit > 0:
          break
    return np.array(labels), texts

In [26]:
train_labels, train_texts = get_labels_and_texts('data/amazon/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('data/amazon/test.ft.txt.bz2')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)

## Task 1

Tokenizing

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

MAX_FEATURES = 12000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

## Task 2

Padding

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)


## Task 3

define the model

In [11]:
from tensorflow.python.keras import models, layers, optimizers

model = models.Sequential()
model.add(layers.Input(shape=(MAX_LENGTH,)))
model.add(layers.Embedding(MAX_FEATURES, 64))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(3))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])

## Task 4

train and evaluate the model

In [13]:
NUM_INSTANCES=50000

history = model.fit(train_texts[:NUM_INSTANCES], train_labels[:NUM_INSTANCES], 
    batch_size=128, epochs=2,
    validation_data=(val_texts, val_labels), )

Epoch 1/2
391/391 [==============================] - 29s 73ms/step - loss: 0.1205 - binary_accuracy: 0.9568 - val_loss: 0.2791 - val_binary_accuracy: 0.9007
Epoch 2/2
391/391 [==============================] - 30s 76ms/step - loss: 0.0648 - binary_accuracy: 0.9790 - val_loss: 1.1615 - val_binary_accuracy: 0.7757


Evaluate the model on unseen test data

In [14]:
model.evaluate(test_texts, test_labels)

3125/3125 [==============================] - 10s 3ms/step - loss: 1.1849 - binary_accuracy: 0.7718: 0s - loss: 1.1827 - binary_accur


[1.184862732887268, 0.7718200087547302]